In [2]:
import numpy as np


import keras
from keras import models
from keras import layers

import tensorflow as tf

In [4]:
X = np.random.normal(0,1,(1000,50))
import numpy.linalg as lin

# the following is to make X more skewed towards a low rank matrix, so the results
# are more instructive. 
u,l,v = lin.svd(X)
wt = [1.1**(-x) for x in range(len(l))]
l = l * wt
print(l)

X = u @ np.vstack((np.diag(l),np.zeros((950,50)))) @ v.T

# m is the number of coordinates of the input examples, 10 is the 
# number of latent coordinates we will use to reconstruct the data
m = 50
k = 10

[38.58364406 34.34619544 30.89081294 27.93062051 24.88376953 22.56866088
 20.32152618 18.40247108 16.58534059 14.93300112 13.45546167 12.0911392
 10.9263694   9.79104276  8.8826086   7.98419025  7.24687719  6.52717597
  5.90523652  5.34885142  4.82116115  4.3672633   3.93922941  3.56590957
  3.22634632  2.90428847  2.61309852  2.360745    2.13125294  1.91731805
  1.73044488  1.56680519  1.40688944  1.27684599  1.15307048  1.03913648
  0.93854756  0.84082882  0.75982894  0.68934112  0.62506331  0.55842443
  0.50028669  0.45083977  0.40380504  0.36174708  0.32585859  0.2893414
  0.26002066  0.23279599]


In [6]:
autoencoder = models.Sequential()
Wlayer = layers.Dense(k,activation='linear', input_shape=(m,))
autoencoder.add(Wlayer)

Vlayer = layers.Dense(m,activation='linear')
# If you want to fix V and see how W varies, uncomment the following line:
# Vlayer.trainable = False
autoencoder.add(Vlayer)

#Vlayer.set_weights([V.T,np.zeros(50,)])
autoencoder.compile(optimizer='rmsprop', loss='mse')
autoencoder.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
32/32 [==============================] - 0s 936us/step - loss: 0.1739
Epoch 2/20
32/32 [==============================] - 0s 694us/step - loss: 0.1335
Epoch 3/20
32/32 [==============================] - 0s 848us/step - loss: 0.1081
Epoch 4/20
32/32 [==============================] - 0s 701us/step - loss: 0.0887
Epoch 5/20
32/32 [==============================] - 0s 862us/step - loss: 0.0736
Epoch 6/20
32/32 [==============================] - 0s 734us/step - loss: 0.0622
Epoch 7/20
32/32 [==============================] - 0s 968us/step - loss: 0.0537
Epoch 8/20
32/32 [==============================] - 0s 784us/step - loss: 0.0475
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0427
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0389
Epoch 11/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0357
Epoch 12/20
32/32 [==============================] - 0s 807us/step - loss: 0.0330
Epoch 13/20
32/32 [============

In [7]:
autoencoder.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0217
Epoch 2/20
32/32 [==============================] - 0s 551us/step - loss: 0.0214
Epoch 3/20
32/32 [==============================] - 0s 858us/step - loss: 0.0211
Epoch 4/20
32/32 [==============================] - 0s 696us/step - loss: 0.0208
Epoch 5/20
32/32 [==============================] - 0s 924us/step - loss: 0.0206
Epoch 6/20
32/32 [==============================] - 0s 833us/step - loss: 0.0204
Epoch 7/20
32/32 [==============================] - 0s 751us/step - loss: 0.0203
Epoch 8/20
32/32 [==============================] - 0s 759us/step - loss: 0.0202
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0201
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0200
Epoch 11/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0199
Epoch 12/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0199
Epoch 13/20
32/32 [================

In [11]:
W = autoencoder.layers[0].get_weights()[0].T
V = autoencoder.layers[1].get_weights()[0].T
print('shape of W: ', W.shape)
print('shape of V: ', V.shape)

shape of W:  (10, 50)
shape of V:  (50, 10)


In [12]:
# Verify that WV =I, the weights W of the first 
# layer is the projection into the column space of the weights of the second layer, V.
# You won't get exactly I because the optimization is stochastic, but close enough

np.round(W @ V,1)

array([[ 1.,  0., -0., -0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0.,  1.,  0.,  0.,  0., -0., -0.,  0., -0., -0.],
       [-0., -0.,  1., -0., -0., -0., -0., -0., -0.,  0.],
       [-0.,  0., -0.,  1.,  0.,  0.,  0., -0.,  0., -0.],
       [ 0.,  0., -0., -0.,  1., -0.,  0.,  0.,  0., -0.],
       [-0., -0., -0.,  0., -0.,  1., -0.,  0., -0., -0.],
       [ 0.,  0., -0., -0., -0.,  0.,  1., -0., -0., -0.],
       [ 0., -0., -0.,  0., -0.,  0.,  0.,  1.,  0., -0.],
       [-0., -0.,  0., -0., -0., -0., -0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -0., -0., -0.,  1.]], dtype=float32)

In [13]:
np.round(V @ W, 1)

array([[ 0.2, -0.1, -0.1, ...,  0.1,  0. , -0. ],
       [-0. ,  0.2,  0.1, ...,  0. ,  0. ,  0. ],
       [-0.1,  0.1,  0.1, ..., -0. , -0. ,  0.1],
       ...,
       [ 0. , -0.1, -0. , ...,  0.4,  0. ,  0.1],
       [ 0. ,  0. , -0. , ..., -0.1,  0.1, -0.1],
       [-0. ,  0. ,  0.1, ...,  0.1, -0. ,  0.2]], dtype=float32)

In [20]:
# We build a new model that ties the weights of the W and V layers (ie forces W=V^T) 
# to force the weight matrices V (W resp) of the prior example to have orthogonal 
# columns, ie V^T V = I (rows resp, W W^T = I) 

# This approximates the SVD. To tie the weights together, we will have to 
# extend the keras.layers.Layer class to take in another layer, and use the weights from 
# that layer. The minimal thing we need to do for this is redefine the output computation 
# when the layer is called, and this is achieved by redefining the inbuilt call function of 
# the Layer class. 

class mylayer(layers.Layer):
    def __init__(self, tracklayer=None, activation=None, **kwargs):
        self.kernel_to_track = tracklayer
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    
    def call(self, inputs):
        output = tf.matmul(inputs, self.kernel_to_track.weights[0], transpose_b = True)
        return self.activation(output)
    


In [21]:
# To define the autoencoder model:

class autoencoder_svd(keras.Model):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.encoder = layers.Dense(10, activation = 'linear')
        self.decoder = mylayer(self.encoder, activation = 'linear')
    
    def call(self, inputs):
        w = self.encoder(inputs)
        self.latent = w
        hatx = self.decoder(w)
        return(hatx)


In [22]:
svdnn = autoencoder_svd()

svdnn.compile(optimizer='adam', loss='mse')
svdnn.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
32/32 [==============================] - 0s 896us/step - loss: 0.1416
Epoch 2/20
32/32 [==============================] - 0s 667us/step - loss: 0.1185
Epoch 3/20
32/32 [==============================] - 0s 814us/step - loss: 0.1022
Epoch 4/20
32/32 [==============================] - 0s 691us/step - loss: 0.0880
Epoch 5/20
32/32 [==============================] - 0s 536us/step - loss: 0.

In [23]:
svdnn.fit(X,X,epochs=20,batch_size=32)


Epoch 1/20
32/32 [==============================] - 0s 806us/step - loss: 0.0262
Epoch 2/20
32/32 [==============================] - 0s 806us/step - loss: 0.0255
Epoch 3/20
32/32 [==============================] - 0s 682us/step - loss: 0.0249
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0243
Epoch 5/20
32/32 [==============================] - 0s 882us/step - loss: 0.0237
Epoch 6/20
32/32 [==============================] - 0s 676us/step - loss: 0.0232
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 0.0227
Epoch 8/20
32/32 [==============================] - 0s 875us/step - loss: 0.0222
Epoch 9/20
32/32 [==============================] - 0s 683us/step - loss: 0.0218
Epoch 10/20
32/32 [==============================] - 0s 691us/step - loss: 0.0215
Epoch 11/20
32/32 [==============================] - 0s 873us/step - loss: 0.0212
Epoch 12/20
32/32 [==============================] - 0s 675us/step - loss: 0.0209
Epoch 13/20
32/32 [==========

In [24]:
W = svdnn.encoder.weights[0].numpy() 
print(W.shape)

np.round(W.T @W,1)

(50, 10)


array([[ 1.,  0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.],
       [ 0.,  1.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0.],
       [-0.,  0.,  1., -0.,  0.,  0., -0., -0.,  0., -0.],
       [-0.,  0., -0.,  1.,  0., -0., -0.,  0., -0., -0.],
       [ 0., -0.,  0.,  0.,  1., -0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -0., -0.,  1.,  0.,  0., -0., -0.],
       [ 0., -0., -0., -0.,  0.,  0.,  1.,  0.,  0., -0.],
       [-0.,  0., -0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  1.,  0.],
       [-0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  1.]], dtype=float32)

In [25]:
# finally, we check how much power the columns of W carry on the top 10 eigen-vectors
# of X^TX, ie the best directions to represent the rows of X.
# The closer these numbers are to 1, the better the column space of W represents the 
# optimal linear space to project the rows of X into. Not all numbers need to be 
# close to 1, but at least some need to be.

V = v[:,0:10]
print(V.shape)


# This is the projection of W into smaller eigen-directions
pwr = lin.inv(V.T @ V) @ V.T @ W

print(np.diag(pwr.T @ pwr))


(50, 10)
[0.95233522 0.99358056 0.95987988 0.9969597  0.9835332  0.99383293
 0.99459085 0.96955484 0.98650035 0.99621449]
